# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [1]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:1


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [5]:
datasets = ['train', 'valid', 'test']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/secureBERT_100/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/secureBERT_100/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/secureBERT_100/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp1-2/training_data/exp_2/secureBERT_100/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

# dataloaders = create_dataloaders(4)
dataloaders = create_dataloaders(16)

- Turn the print message to a log file

In [7]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_secureBERT_100.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0918_19:10_GraphSAGE_secureBERT_100.log


### Model

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')  # Output layer

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.layer2(g, h)  # You can apply another non-linearity here if needed
        
        g.ndata['h'] = h
        hg = dgl.mean_nodes(g, 'h')
        return hg


- Model Forward  

In [9]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
#     labels = labels.float()

    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
#     logits = logits.mean(dim=1)
#     print(labels)
#     print(labels.shape)
#     print(logits)
#     print(logits.shape)

    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1) # for GAT
#     print(preds)
#     preds = logits.argmax() # for graphSAGE
    
    '''
    问题出在GAT和GraphSAGE模型的输出形状上。
    在你的 GAT 模型中，最后一层的输出是一个一维的张量，因此 logits 是一个形状为 (batch_size,) 的张量。这是因为 GAT 模型最后一层的输出被处理成了一个节点的预测，而不是整个图的预测。
    在 GraphSAGE 模型中，最后一层的输出形状是 (num_nodes, out_dim)，也就是说，它返回了整个图的节点级别的预测。因此，logits 是一个二维的张量，形状为 (num_nodes, out_dim)。
    当你尝试在一个一维张量上调用 logits.argmax(1) 时，会产生维度错误，因为它期望一个二维张量来执行操作。
    所以，在 GraphSAGE 模型中，你应该将以下行：
    '''
    
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [10]:
seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=100, hidden_dim=16, out_dim=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [11]:
model.layer1.fc_self.weight

Parameter containing:
tensor([[-0.2663, -0.2455, -0.0843,  ..., -0.2211,  0.2500, -0.1713],
        [ 0.1325, -0.1434, -0.2196,  ...,  0.1400,  0.2420,  0.1997],
        [ 0.0983,  0.2508,  0.0598,  ..., -0.1050, -0.1366,  0.2606],
        ...,
        [-0.2861,  0.2042, -0.2862,  ..., -0.1234,  0.3175, -0.2434],
        [ 0.3105,  0.2460,  0.1613,  ...,  0.2424, -0.3065, -0.3124],
        [ 0.0262,  0.2473,  0.3131,  ..., -0.0755, -0.3011, -0.0491]],
       requires_grad=True)

- Check if model really load the model_dict

In [12]:
model = GraphSAGE(in_dim=100, hidden_dim=16, out_dim=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.layer1.fc_self.weight

Parameter containing:
tensor([[-0.2663, -0.2455, -0.0843,  ..., -0.2211,  0.2500, -0.1713],
        [ 0.1325, -0.1434, -0.2196,  ...,  0.1400,  0.2420,  0.1997],
        [ 0.0983,  0.2508,  0.0598,  ..., -0.1050, -0.1366,  0.2606],
        ...,
        [-0.2861,  0.2042, -0.2862,  ..., -0.1234,  0.3175, -0.2434],
        [ 0.3105,  0.2460,  0.1613,  ...,  0.2424, -0.3065, -0.3124],
        [ 0.0262,  0.2473,  0.3131,  ..., -0.0755, -0.3011, -0.0491]],
       requires_grad=True)

In [ ]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GraphSAGE(in_dim=100, hidden_dim=16, out_dim=167)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_secureBERT_100.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

total_steps = 10

# save the best model
best_val_loss = float('inf')
patience = 10  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
#     scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

#         print(best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/18/2023, 19:12:53# labels of 5000: tensor([ 94,  77,  81,  86, 161,  61,  45,  21,   7,  27,  60,  37,  74,  46,
        145,  33], device='cuda:1') torch.Size([16])
09/18/2023, 19:12:53# predicted of 5000: tensor([ 94,  77, 109,  86, 161,  61,  39, 166,   7,  79,  49,  37,  33,  46,
        145,  49], device='cuda:1') torch.Size([16])


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



09/18/2023, 19:18:00# labels of 5000: tensor([ 48, 111, 153, 123,  68, 111, 104,   8,  18, 149,  23,  57,  71,  89,
          1,  42], device='cuda:1') torch.Size([16])
09/18/2023, 19:18:00# predicted of 5000: tensor([ 49, 100, 153, 123,  88,  49,  87, 103,  87, 149,  23,  58,  71,  89,
         31,  47], device='cuda:1') torch.Size([16])


...

09/18/2023, 19:35:50# total batches: 8300
09/18/2023, 19:35:50# Epoch 4 | Train Loss: 1.5185 | Train Accuracy: 0.6095


Validation:   0%|          | 0/1038 [00:00<?, ?it/s]

09/18/2023, 19:35:51# labels of Validation: tensor([ 62,  66, 118, 145, 122,  89,  76, 109,  51, 103, 151, 117,  64, 100,
        159, 108], device='cuda:1') torch.Size([16])
09/18/2023, 19:35:51# predicted of Validation: tensor([ 62,  66, 118, 145, 122,  89, 157, 157,  51, 110,  54, 117,  88, 100,
        159, 131], device='cuda:1') torch.Size([16])
09/18/2023, 19:35:51# labels of 0: tensor([ 62,  66, 118, 145, 122,  89,  76, 109,  51, 103, 151, 117,  64, 100,
        159, 108], device='cuda:1') torch.Size([16])
09/18/2023, 19:35:51# predicted of 0: tensor([ 62,  66, 118, 145, 122,  89, 157, 157,  51, 110,  54, 117,  88, 100,
        159, 131], device='cuda:1') torch.Size([16])
09/18/2023, 19:36:19# labels of Validation: tensor([165,  72,  52,  99,  52,  13,  60, 159, 153,  17,  10, 158, 156,  66,
         81,  14], device='cuda:1') torch.Size([16])
09/18/2023, 19:36:19# predicted of Validation: tensor([165,  72,  52,  99,  52,  13, 157, 159, 153,  17,  10, 157, 156,  66,
        150,

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/18/2023, 19:39:13# labels of 5000: tensor([153, 109,  69, 157,  52,  23, 139,  20,  43, 108,  57,  46, 152, 150,
        122,  74], device='cuda:1') torch.Size([16])
09/18/2023, 19:39:13# predicted of 5000: tensor([153,  31,  69,  71,  52,  23, 139,  20,  43, 108, 121,  46, 111,  33,
        122, 121], device='cuda:1') torch.Size([16])


### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [15]:
# load the pretrained model
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_secureBERT_100.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-secureBERT_100-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-secureBERT_100-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/1038 [00:00<?, ?it/s]

09/18/2023, 20:01:23# labels of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:23# predicted of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:23# labels of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:23# predicted of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:23# labels: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:23# predicted: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/18/2023, 20:01:42# labels of Test: tensor([162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/18/2023, 20:01:50# report path: classification_report/classification_report-secureBERT_100-graphSAGE-1.xlsx
09/18/2023, 20:01:50# label path: classification_report/mapped_true_predicted_labels-secureBERT_100-graphSAGE-1.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/18/2023, 20:01:50# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00       100
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00       100
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00       100
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00       100
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.00      0.00      0.00       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       100
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00       100
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.00      0.00      0.00       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.62      0.05      0.09     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
